## 한국 경마 2014년도 데이터

- 데이터셋 다운로드
    - https://www.kaggle.com/parksami/koreahorseracedata
- 순위를 승점제로 했을때 2014년 시즌에서 말들의 성적은?

##### Race load dataset

In [61]:
import pandas as pd
import numpy as np

In [73]:
df_race = pd.read_csv("2014_p.csv", encoding="euc-kr")
df_race.tail(2)

,ID,RCTRCK,RACE_DE,RACE_NO,PARTCPT_NO,RANK,RCHOSE_NM,HRSMN,RCORD,ARVL_DFFRNC,EACH_SCTN_PASAGE_RANK,A_WIN_SYTM_EXPECT_ALOT,WIN_STA_EXPECT_ALOT
27216,27217,제주,2014-11-29,9,7,6.0,미주여행,김경휴,0:01:31.1,13,2 - - - 2 - 3 - 6,6.2,9.4
27217,27218,제주,2014-11-29,9,6,1.0,철옹성,장우성,0:01:26.6,NaN,1 - - - 1 - 1 - 1,3.9,2.9


##### Horse load dataset

In [74]:
df_horse = pd.read_csv('2014_s.csv', encoding='euc-kr')
df_horse.tail(2)

,ID,RCTRCK,RACE_DE,PRDCTN_NATION_NM,SEX,AGE,BND_WT,TRNER,RCHOSE_OWNR_NM,RCHOSE_BDWGH
27216,27217,제주,2014-11-29,한,거,NaN,53.0,강대은,김기준,281
27217,27218,제주,2014-11-29,한,거,NaN,57.5,박병진,강상우,314


### Quiz 1. 어떤 성별을 가진 말이 성적이 좋을까?

- race 데이터 프레임 : 성적데이터
- horse 데이터 프레임 : 성별 데이터
- race 데이터와 horse 데이터를 ID를 key값으로 merge

- 분석 절차
    - 1. 성별에 따른 순위 데이터 프레임으로 생성
    - 2. 전체 데이터에서 성별 데이터를 나눠서 확률 데이터로 변경
    - 3. 1 ~ 5등까지의 성별에 따른 등수 확률 데이터를 출력

In [75]:
df_merged = pd.merge(df_race, df_horse)
df_merged.tail(3)

,ID,RCTRCK,RACE_DE,RACE_NO,PARTCPT_NO,RANK,RCHOSE_NM,HRSMN,RCORD,ARVL_DFFRNC,EACH_SCTN_PASAGE_RANK,A_WIN_SYTM_EXPECT_ALOT,WIN_STA_EXPECT_ALOT,PRDCTN_NATION_NM,SEX,AGE,BND_WT,TRNER,RCHOSE_OWNR_NM,RCHOSE_BDWGH
27215,27216,제주,2014-11-29,4,3,7.0,산정무한,안득수,0:01:22.8,1½,4 - - - 4 - 4 - 6,30.9,5.2,한,거,NaN,52.5,김영래,광명종합,333
27216,27217,제주,2014-11-29,9,7,6.0,미주여행,김경휴,0:01:31.1,13,2 - - - 2 - 3 - 6,6.2,9.4,한,거,NaN,53.0,강대은,김기준,281
27217,27218,제주,2014-11-29,9,6,1.0,철옹성,장우성,0:01:26.6,NaN,1 - - - 1 - 1 - 1,3.9,2.9,한,거,NaN,57.5,박병진,강상우,314


In [76]:
# 1. 성별에 따른 순위 데이터 프레임으로 생성
df_merged['COUNT'] = 1
df = df_merged.pivot_table("COUNT", 'RANK', 'SEX', aggfunc=np.sum)
df.head()

SEX,거,수,암
RANK,,,
1.0,528.0,1067.0,964.0
2.0,488.0,1033.0,1021.0
3.0,538.0,984.0,1029.0
4.0,524.0,947.0,1082.0
5.0,502.0,897.0,1148.0


In [77]:
# 성별에 대한 전체 데이터
df_merged.groupby("SEX")['COUNT'].sum()

SEX
거     5576
수     9869
암    11773
Name: COUNT, dtype: int64

In [78]:
# 2. 전체 데이터에서 성별 데이터를 나눠서 확률 데이터로 변경
df = df / df_merged.groupby("SEX")['COUNT'].sum() * 100
df.head()

SEX,거,수,암
RANK,,,
1.0,9.469154,10.811632,8.188227
2.0,8.751793,10.467119,8.672386
3.0,9.648494,9.970615,8.740338
4.0,9.397418,9.595704,9.190521
5.0,9.002869,9.089067,9.751125


In [79]:
# 소수 둘째자리까지만 출력
# 결론 : 수컷이 좋은 성적을 낼 확률이 높다.
df = round(df, 2)
df.head()

SEX,거,수,암
RANK,,,
1.0,9.47,10.81,8.19
2.0,8.75,10.47,8.67
3.0,9.65,9.97,8.74
4.0,9.40,9.60,9.19
5.0,9.00,9.09,9.75


###  Quiz 2. 순위를 승점제로 했을때 2014년 시즌에서 말들의 성적은?

- 1등 : 3점
- 2등 : 2점
- 3등 : 1점

In [90]:
def rank_point(rank):
    if rank == 1.0: return 3
    elif rank == 2.0: return 2
    elif rank == 3.0: return 1
    else: return 0

df_merged['POINT'] = df_merged['RANK'].apply(rank_point)

In [97]:
df_merged['POINT'] = df_merged['RANK'].apply(lambda rank: 4-rank if rank <= 3.0 else 0)

In [98]:
df_merged.groupby('RCHOSE_NM')['POINT'].sum().sort_values(ascending=False)[:10]

RCHOSE_NM
신천보스      25.0
금포스카이     24.0
가리사니      23.0
한강의기적     23.0
신천돌채      22.0
원일강자      21.0
추종불허      21.0
늘푸른타이거    21.0
스트롱로드     21.0
와신상담      21.0
Name: POINT, dtype: float64